## 1. Latih model dasar (baseline model)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Baseline model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_split=0.1)
loss, accuracy = model.evaluate(x_test, y_test)
print(f"\nAkurasi pada data uji: {accuracy:.4f}")


Epoch 1/2
1688/1688 [==============================] - 3s 1ms/step - loss: 0.2717 - accuracy: 0.9233 - val_loss: 0.1233 - val_accuracy: 0.9660
Epoch 2/2
313/313 [==============================] - 0s 816us/step - loss: 0.1081 - accuracy: 0.9681

Akurasi pada data uji: 0.9681


## 2. Pruning model menggunakan TensorFlow Model Optimization Toolkit

In [2]:
import tensorflow_model_optimization as tfmot

# Define pruning schedule
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=len(x_train) // 128 * 2  # 2 epochs
    )
}

# Apply pruning
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Re-compile and train
pruned_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

pruned_model.fit(x_train, y_train,
                 epochs=2,
                 batch_size=128,
                 validation_split=0.1,
                 callbacks=[tfmot.sparsity.keras.UpdatePruningStep()])
loss, accuracy = pruned_model.evaluate(x_test, y_test)
print(f"\nAkurasi pada data uji: {accuracy:.4f}")

Epoch 1/2
422/422 [==============================] - 2s 2ms/step - loss: 0.0754 - accuracy: 0.9776 - val_loss: 0.0813 - val_accuracy: 0.9757
Epoch 2/2
313/313 [==============================] - 0s 949us/step - loss: 0.0778 - accuracy: 0.9762

Akurasi pada data uji: 0.9762


## 3. Post-Training Quantization (PTQ)

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Aktivasi quantisasi
tflite_quant_model = converter.convert()

# Simpan model hasil quantization
with open('model_pruned_quantized.tflite', 'wb') as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: C:\Users\agung\AppData\Local\Temp\tmph5car9qt\assets


INFO:tensorflow:Assets written to: C:\Users\agung\AppData\Local\Temp\tmph5car9qt\assets


## 4. Cek Ukuran Model Sebelum dan Sesudah

In [4]:
import os

# Save original model
model.save('original_model.h5')

# Ukuran file
print("Original Model Size (MB):", os.path.getsize("original_model.h5") / 1e6)
print("Pruned + Quantized Model Size (MB):", os.path.getsize("model_pruned_quantized.tflite") / 1e6)


Original Model Size (MB): 1.247056
Pruned + Quantized Model Size (MB): 0.818276


c:\Users\agung\miniconda3\envs\embedded_ai\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
